In [ ]:
%load_ext autoreload
%autoreload 2

from warnings import filterwarnings

import torch
import torch.nn as nn

import config_iProL as config
from dataset import get_datasets, get_cv_dataloader_list
from dataset import get_excel, get_complete_data
from module import Longformer_base_Net
from tools import check_result_dir
from tools_fit import model_fit

In [ ]:
sheet = get_excel(version='', sheet_name='dataset')
complete_data = get_complete_data(sheet)

dataset_identify, _, _ = get_datasets(complete_data)

identify_dataset_list, identify_loader_list = get_cv_dataloader_list(config, dataset_identify, "Pro")

In [ ]:
filterwarnings("ignore")
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)

In [ ]:
"""
识别promoter：Pro
"""
for cv_idx, (train_loader, test_loader) in enumerate(identify_loader_list, 1):
    check_result_dir(config, cv_idx)
    # model = Longformer_base_lstm_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """去掉一个CNN，dense1参数 in_channels进入"""
    # model = Longformer_base_2cnn_lstm_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """Longformer_base_lstm_BNpos_Net: 修改BN到pooling之后dropout之前， 美纱用"""
    # model = Longformer_base_lstm_BNpos_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """
    没有LSTM
    """
    model = Longformer_base_Net(config.model_name, config.device,
                                     config.add_special_tokens)
    model.to(config.device)

    # 这里是一般情况，共享层往往不止一层，所以做一个for循环
    if hasattr(model, 'longformer') and config.not_fine_tuning:
        for para in model.longformer.parameters():
            para.requires_grad = False
        # print(module.parameters())

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.6)
    criterion = nn.BCELoss()

    model_fit(config, model, train_loader, test_loader, optimizer, criterion, scheduler, 0.86, cv_idx)


In [ ]:
import time

now_time = time.strftime('%m%d%H%M%S', time.localtime())
print(f"『now_time』:{now_time}")